<a href="https://colab.research.google.com/github/1722101709/Internship/blob/Emergence/pnas_1314922110_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Importing Modules***

In [ ]:
import pandas as pd # pandas for storing the data in DataFrame
from scipy.special import rel_entr # KL Divergence(relative entropy or negation of entropy)

# ***Probability Transition Matrix for internal states in micro states***

In [ ]:
n = 4
# no of bits used to represent the micro state

ptm = [[0.7,0.3], [0.7,0.3], [0.7,0.3], [0,1]]
# probability transition matrix of micro elements

micro_ptm = pd.DataFrame(ptm,columns = ['0','1'], index=['00','01','10','11'])
# representing the micro elements probability transition matrix in DataFrame format

micro_ptm

,0,1
00,0.7,0.3
01,0.7,0.3
10,0.7,0.3
11,0.0,1.0


# ***Probability Transition Matrix for internal states in macro states***

In [ ]:
ptm = [[0.81,0.19], [0,1]]
# probability transition matrix of macro elements

macro_ptm = pd.DataFrame(ptm,columns = ['off','on'], index=['off','on'])
# representing the macro elements probability transition matrix in DataFrame format

macro_ptm

,off,on
off,0.81,0.19
on,0.00,1.00


# ***Methods to find difference/similarity between the probabilities***

In [ ]:
# KL Divergence
def kldivergence():
    return lambda x,y: rel_entr(x,y)

# Kmeans with Euclidean distance
def kmeans():
    return lambda x,y: abs(x-y)

# ***prediction of future states at micro level***

In [ ]:
# finding Effective Information(EI) of micro states
def micro(n,diff):
    ei_micro = 0 # initializing ei to 0
    ptm = [] # initializing probability transition matrix of micro states
    N = 2**n # size of state space
    y = '{0:0'+str(n)+'b}' # binary formatter to format integer to binary
    for i in range(N): # i from 0 to 2**n-1 which are all possible states
        s0 = y.format(i) # finding the current binary state
        temp = [] # to store the probailities of all possible states to be a future state given the current state s0
        for j in range(N): # looping over all possible future states given current state s0
            sf = y.format(j) # formating the future state into binary format
            s = s0[-2:] + sf
            p = 1 # initializing probability of sf to be future state given current state with contrained distribution effect/cause to 1
            q = 1/N # probability of sf to be future state given current state with uncontrained distribution effect/cause = 1/N since uniformly distributed
            for k in range(n): # finding the probability by looping over all possible micro elements
                p *= micro_ptm[s[k+2]][s[k:k+2]]
            ei_micro += diff(p,q) # ei = difference between the probability with constrained(p) and unconstrained distributions(q)
            temp.append(p)
        ptm.append(temp)
    ei_micro /= N # as state space is N, ei for one state = ei/N
    print(f'EI = {ei_micro}') # printing ei value
    print(f'Effectiveness = {round(ei_micro/n*100,2)}%') # printing effectiveness of prediction

    print('\nProbability Transition matrix for micro states is')
    ptm = pd.DataFrame(ptm, columns = [y.format(i) for i in range(N)], index = [y.format(i) for i in range(N)]) # storing the Probability transition values of macro states in DataFrame
    return ptm

# ***prediction of future states at macro level***

In [ ]:
# finding Effective Information(EI) of macro states
def macro(n,diff):
    ei_macro = 0 # initializing ei to 0
    PTM = [] #  initializing probability transition matrix of macro states
    d = {'00':'off','01':'off','10':'off','11':'on'} # mapping of micro elements to macro elements
    y = '{0:0'+str(n)+'b}' # binary formatter to format integer to binary
    for i in range(n): # i from 0 to n which are all possible states
        temp = [] # to store the probailities of all possible states to be a future state given the current state s0
        for j in range(n):
            s = y.format(i*n+j) # finding the current binary state
            p = 1 # initializing probability of sf to be future state given current state with contrained distribution effect/cause to 1
            q = 1/n # probability of sf to be future state given current state with uncontrained distribution effect/cause = 1/N since uniformly distributed
            for k in range(n-2): # finding the probability by looping over all possible macro elements
                p *= macro_ptm[d[s[k+1:k+3]]][d[s[k:k+2]]]
            ei_macro += diff(p,q) # ei = difference between the probability with constrained(p) and unconstrained distributions(q)
            temp.append(p)
        PTM.append(temp)
    ei_macro /= n # as state space is n, ei for one state = ei/n
    print(f'EI with macro states = {ei_macro}') # printing ei value
    print(f'effectiveness = {round((2*ei_macro/n)*100,2)}%') # printing effectiveness
    y = '{0:0'+str(n//2)+'b}'
    PTM = pd.DataFrame(PTM, columns = [y.format(i) for i in range(n)], index = [y.format(i) for i in range(n)]) # storing the Probability transition values of macro states in DataFrame
    print(f'\nProbability transition matrix :-')
    return PTM

# ***EI at micro level using KL Divergence***

In [ ]:
micro(n,kldivergence())

EI = 1.1612814094944874
Effectiveness = 29.03%

Probability Transition matrix for micro states is


,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0000,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0001,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30
0010,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0011,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.00
0100,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0101,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30
0110,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0111,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.00
1000,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
1001,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30


# ***EI at micro level using KMeans***

In [ ]:
micro(n,kmeans())

EI = 1.1043499999999997
Effectiveness = 27.61%

Probability Transition matrix for micro states is


,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0000,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0001,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30
0010,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0011,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.00
0100,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0101,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30
0110,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
0111,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.00
1000,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.1029,0.0441,0.0441,0.0189,0.0,0.0,0.0,0.09
1001,0.2401,0.1029,0.1029,0.0441,0.1029,0.0441,0.0,0.063,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.30


# ***EI at macro level using KL Divergence***

In [ ]:
macro(n,kldivergence())

EI with macro states = 1.5622841275493713
effectiveness = 78.11%

Probability transition matrix :-


,00,01,10,11
00,0.6561,0.6561,0.6561,0.1539
01,0.6561,0.6561,0.0000,0.1900
10,0.6561,0.6561,0.6561,0.1539
11,0.0000,0.0000,0.0000,1.0000


# ***EI at macro level using KMeans***

In [ ]:
macro(n,kmeans())

EI with macro states = 1.3127500000000003
effectiveness = 65.64%

Probability transition matrix :-


,00,01,10,11
00,0.6561,0.6561,0.6561,0.1539
01,0.6561,0.6561,0.0000,0.1900
10,0.6561,0.6561,0.6561,0.1539
11,0.0000,0.0000,0.0000,1.0000
